# Import Statements

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from multiprocessing import Pool, cpu_count
import keras
from keras.models import Sequential, Input, Model
from keras.layers import Dense, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Flatten, Dropout
from keras.layers import BatchNormalization, LSTM, Merge, Concatenate, merge, Reshape
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras import regularizers
from keras import backend as K
from matplotlib.pyplot import imshow
from scipy.signal import decimate
import pywt
from sklearn.metrics import f1_score
import seaborn as sn

# Data Extraction

In [ ]:
pa = pd.read_csv("Phones_accelerometer.csv")
# pa=pd.read_csv("Phones_gyroscope.csv")
# wa=pd.read_csv("Watch_accelerometer.csv")
# wg=pd.read_csv("Watch_gyroscope.csv")
print (pa.shape)
print("Done")

In [ ]:
pa = pa[pa['gt'] != 'null']
acts = pa['gt'].unique()
users = pa['User'].unique()
devices = pa['Device'].unique()
print(devices)
print (acts)
print (users)

In [ ]:
l = {}
for i, act in enumerate(acts):
    l[act] = i
print (l)

dev = {}
for i, d in enumerate(devices):
    dev[d] = i
print (dev)

use = {}
for i, u in enumerate(users):
    use[u] = i
print (use)

In [ ]:
devs, acc, labels = [], [], []

for act in acts:
    pa_act = pa[pa['gt'] == act]
    for device in devices:
        pa_dev = pa_act[(pa_act['Device'] == device)]
        for user in users:
            pa_user = pa_dev[pa_dev['User'] == user]
            pa_user = pa_user[['x', 'y','z']]
            if str(device) == 'nexus4_1' or str(device) == 'nexus4_2':
                min_win = 400
            elif str(device) == 's3_1' or str(device) == 's3_2':
                min_win = 300
            elif str(device) == 's3mini_1' or str(device) == 's3mini_2':
                min_win = 200
            else:
                min_win = 100
            if(pa_user.shape[0] >= min_win):
                acc.append(pa_user.values)
                devs.append(device)
                labels.append(l[act])
    print (f'{act} done')
    
acc = np.array(acc)
labels = np.array(labels)
devs = np.array(devs)
print ("Done")

print(acc.shape, labels.shape, devs.shape)

In [ ]:
acc[390].shape

# Getting the Windowed Data

In [ ]:
def getWindowedData(index, w_min):
    
    windowData, windowLabels = [], []
    num_windows = acc[index].shape[0] // w_min
    if num_windows == 0:
        print(acc[index].shape[0], w_min)
    k = 0
    for _ in range(num_windows):
        windowData.append(acc[index][k:k+w_min])
        k += w_min
        windowLabels.append(labels[index])
    return windowData, windowLabels

In [ ]:
# Getting 2 seconds (100 samples) of data for all devices

windowedData = []
for i in range(len(acc)):
    if str(devs[i]) == 'nexus4_1' or str(devs[i]) == 'nexus4_2':
        w_min = 400
    elif str(devs[i]) == 's3_1' or str(devs[i]) == 's3_2':
        w_min = 300
    elif str(devs[i]) == 's3mini_1' or str(devs[i]) == 's3mini_1':
        w_min = 200
    else:
        w_min = 100
    windowedData.append((getWindowedData(i, w_min)))

In [ ]:
np.array(windowedData).shape

In [ ]:
windowedData = np.array(windowedData)
print (windowedData.shape)

# Decimating the Windowed Data

In [ ]:
def decimateThatSignal(i):
    decimatedSignalData, decimatedSignalLabels = [], []
    
    if windowedData[i][0][0].shape[0] == 400:
        for j in range(len(windowedData[i][0])):
            decimatedX = decimate(windowedData[i][0][j][:,0], 4, zero_phase=True)
            decimatedY = decimate(windowedData[i][0][j][:,1], 4, zero_phase=True)
            decimatedZ = decimate(windowedData[i][0][j][:,2], 4, zero_phase=True)
            decimatedSignal = np.dstack((decimatedX, decimatedY, decimatedZ))
            decimatedSignalData.append(decimatedSignal)
            decimatedSignalLabels.append(windowedData[i][1][j])
        return np.array(decimatedSignalData), np.array(decimatedSignalLabels)
    
    elif windowedData[i][0][0].shape[0] == 300:
        for j in range(len(windowedData[i][0])):
            decimatedX = decimate(windowedData[i][0][j][:,0], 3, zero_phase=True)
            decimatedY = decimate(windowedData[i][0][j][:,1], 3, zero_phase=True)
            decimatedZ = decimate(windowedData[i][0][j][:,2], 3, zero_phase=True)
            decimatedSignal = np.dstack((decimatedX, decimatedY, decimatedZ))
            decimatedSignalData.append(decimatedSignal)
            decimatedSignalLabels.append(windowedData[i][1][j])
        return np.array(decimatedSignalData), np.array(decimatedSignalLabels)
    
    elif windowedData[i][0][0].shape[0] == 200:
        for j in range(len(windowedData[i][0])):
            decimatedX = decimate(windowedData[i][0][j][:,0], 2, zero_phase=True)
            decimatedY = decimate(windowedData[i][0][j][:,1], 2, zero_phase=True)
            decimatedZ = decimate(windowedData[i][0][j][:,2], 2, zero_phase=True)
            decimatedSignal = np.dstack((decimatedX, decimatedY, decimatedZ))
            decimatedSignalData.append(decimatedSignal)
            decimatedSignalLabels.append(windowedData[i][1][j])
        return np.array(decimatedSignalData), np.array(decimatedSignalLabels)
    
    else:
        return np.array(windowedData[i][0]), np.array(windowedData[i][1])

In [ ]:
decimateThatSignal(0)[0].shape

In [ ]:
w_min = 100

decimatedData = decimateThatSignal(0)[0].reshape((-1, w_min, 3))
decimatedLabels = decimateThatSignal(0)[1]
for i in range(1, len(windowedData)):
    print (i)
    decimatedData = np.vstack((decimatedData, decimateThatSignal(i)[0].reshape((-1, w_min, 3))))
    decimatedLabels = np.hstack((decimatedLabels, decimateThatSignal(i)[1]))
decimatedData = np.array(decimatedData)
decimatedLabels = np.array(decimatedLabels)

In [ ]:
print (decimatedData.shape, decimatedLabels.shape)

# Getting the DWTed Data

In [ ]:
pywt.wavelist()

In [ ]:
# Change window size here as and when DWT wavelet changes

w_min = 50+3

In [ ]:
DWTData = []
for i in range(len(decimatedData)):
    Xca, Xda = pywt.dwt(decimatedData[i].reshape((-1, 3))[:,0], wavelet='db4', mode='periodic')
    Yca, Yda = pywt.dwt(decimatedData[i].reshape((-1, 3))[:,1], wavelet='db4', mode='periodic')
    Zca, Zda = pywt.dwt(decimatedData[i].reshape((-1, 3))[:,2], wavelet='db4', mode='periodic')
    coef = np.hstack((Xca, Yca, Zca)).reshape((-1, w_min, 3))
    DWTData.append((coef, decimatedLabels[i]))
    print (i)

DWTData = np.array(DWTData)

In [ ]:
print(DWTData.shape)

In [ ]:
a = 46548
a = 34509
print (DWTData[a][0][0].shape)
plt.plot(decimatedData[a])
plt.ylabel('Inertial g-values')
plt.xlabel(r'Data points in a single $w_a$')
# plt.legend(['X-axis','Y-axis','Z-axis'],loc=1)
plt.legend(['X-axis','Y-axis','Z-axis'],bbox_to_anchor=(0., 1.02, 1., .102), loc=1,ncol=3, mode="expand", borderaxespad=0.)
plt.savefig('plotBeforeDWT.png')
plt.show()
plt.plot(DWTData[a][0][0])
plt.ylabel('Inertial g-values')
plt.xlabel(r'Data points in a single $w_a$')
plt.legend(['X-axis','Y-axis','Z-axis'],bbox_to_anchor=(0., 1.02, 1., .102), loc=1,ncol=3, mode="expand", borderaxespad=0.)
# plt.legend(['X-axis','Y-axis','Z-axis'],loc=1)
plt.savefig('plotAfterDWT.png')
plt.show()
# np.save('beforeDWT',DWTData[a][0][0])
# np.save('afterDWT',decimatedData[a])

In [ ]:
labels = []
data = np.zeros((DWTData.shape[0], 1, w_min, 3))
for i in range(DWTData.shape[0]):
    data[i, :, :] = DWTData[i][0][:]
    labels.append(DWTData[i][1])
    
data = data.reshape((-1, w_min, 3)).astype('float32')
labels = np.array(labels).astype('float32')

print (data.shape, labels.shape)

# Train-Test Split and Normalization

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(data, labels, stratify=labels, test_size=0.2, random_state=5233)

In [ ]:
# Run only once

ytrain = to_categorical(ytrain, len(acts))
ytest = to_categorical(ytest, len(acts))

In [ ]:
print (Xtrain.shape, Xtest.shape, ytrain.shape, ytest.shape)

In [ ]:
np.save('Xtrain_53.npy', Xtrain)
np.save('Xtest_53.npy', Xtest)

In [ ]:
# Delete if necessary

# del acc, pa, windowedData, decimatedData, decimatedLabels, data, labels, DWTData

In [ ]:
Xtrain[:, :, 0].shape

In [ ]:
# Standard Scaler

Xtrain_fit0 = StandardScaler().fit(Xtrain[:, :, 0])
Xtrain0 = Xtrain_fit0.transform(Xtrain[:, :, 0])

Xtrain_fit1 = StandardScaler().fit(Xtrain[:, :, 1])
Xtrain1 = Xtrain_fit1.transform(Xtrain[:, :, 1])

Xtrain_fit2 = StandardScaler().fit(Xtrain[:, :, 2])
Xtrain2 = Xtrain_fit2.transform(Xtrain[:, :, 2])

Xtest0 = Xtrain_fit0.transform(Xtest[:, :, 0])

Xtest1 = Xtrain_fit1.transform(Xtest[:, :, 1])

Xtest2 = Xtrain_fit2.transform(Xtest[:, :, 2])

print (Xtrain0.shape, Xtest0.shape)

In [ ]:
X_train = np.dstack((Xtrain0, Xtrain1, Xtrain2))
X_test = np.dstack((Xtest0, Xtest1, Xtest2))
print (X_train.shape, X_test.shape)

In [ ]:
del Xtrain0, Xtrain1, Xtrain2, Xtest0, Xtest1, Xtest2

In [ ]:
print (np.min(X_train), np.max(X_train), np.min(X_test), np.max(X_test))

In [ ]:
X_train = X_train.reshape((-1, w_min, 3, 1))
X_test = X_test.reshape((-1, w_min, 3, 1))

In [ ]:
print (X_train.shape)
print (X_test.shape)
print (ytrain.shape)
print (ytest.shape)

# Define params for model

In [ ]:
num_classes = len(acts)
learning_rate = 2e-4

# Conv1D (Subnet) -> Conv2D

In [ ]:
X_train0 = X_train[:, :, 0].reshape((-1, w_min, 1))
X_train1 = X_train[:, :, 1].reshape((-1, w_min, 1))
X_train2 = X_train[:, :, 2].reshape((-1, w_min, 1))

X_test0 = X_test[:, :, 0].reshape((-1, w_min, 1))
X_test1 = X_test[:, :, 1].reshape((-1, w_min, 1))
X_test2 = X_test[:, :, 2].reshape((-1, w_min, 1))

print (X_train0.shape, X_test0.shape)

In [ ]:
inputX = Input(shape=(X_train0.shape[1], X_train0.shape[2]))

convX1 = Conv1D(filters=8, kernel_size=2, padding='causal', activation='relu')(inputX)
batchX1 = BatchNormalization()(convX1)
poolX1 = MaxPooling1D(pool_size=2, padding='same')(batchX1)

convX2 = Conv1D(filters=16, kernel_size=2, padding='causal', activation='relu')(poolX1)
batchX2 = BatchNormalization()(convX2)
poolX2 = MaxPooling1D(pool_size=2, padding='same')(batchX2)

# convX3 = Conv1D(filters=128, kernel_size=2, padding='causal', activation='relu')(poolX2)
# batchX3 = BatchNormalization()(convX3)
# poolX3 = MaxPooling1D(pool_size=2, padding='same')(batchX3)

modelX = Flatten()(poolX2)

inputY = Input(shape=(X_train1.shape[1], X_train2.shape[2]))

convY1 = Conv1D(filters=8, kernel_size=2, padding='causal', activation='relu')(inputY)
batchY1 = BatchNormalization()(convY1)
poolY1 = MaxPooling1D(pool_size=2, padding='same')(batchY1)

convY2 = Conv1D(filters=16, kernel_size=2, padding='causal', activation='relu')(poolY1)
batchY2 = BatchNormalization()(convY2)
poolY2 = MaxPooling1D(pool_size=2, padding='same')(batchY2)

# convY3 = Conv1D(filters=128, kernel_size=2, padding='causal', activation='relu')(poolY2)
# batchY3 = BatchNormalization()(convY3)
# poolY3 = MaxPooling1D(pool_size=2, padding='same')(batchY3)

modelY = Flatten()(poolY2)

inputZ = Input(shape=(X_train2.shape[1], X_train2.shape[2]))

convZ1 = Conv1D(filters=8, kernel_size=2, padding='causal', activation='relu')(inputZ)
batchZ1 = BatchNormalization()(convZ1)
poolZ1 = MaxPooling1D(pool_size=2, padding='same')(batchZ1)

convZ2 = Conv1D(filters=16, kernel_size=5, padding='causal', activation='relu')(poolZ1)
batchZ2 = BatchNormalization()(convZ2)
poolZ2 = MaxPooling1D(pool_size=2, padding='same')(batchZ2)

# convZ3 = Conv1D(filters=128, kernel_size=2, padding='causal', activation='relu')(poolZ2)
# batchZ3 = BatchNormalization()(convZ3)
# poolZ3 = MaxPooling1D(pool_size=2, padding='same')(batchZ3)

modelZ = Flatten()(poolZ2)

In [ ]:
merged_model = merge([modelX, modelY, modelZ], mode='concat')
print (K.int_shape(merged_model))

final_merge = Reshape((K.int_shape(merged_model)[1]//3, 3, 1))(merged_model)
print (K.int_shape(final_merge))

conv1 = Conv2D(filters=8, kernel_size=(3, 3), padding='same')(final_merge)
batch1 = BatchNormalization()(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2), padding='same')(batch1)

conv2 = Conv2D(filters=16, kernel_size=(3, 3), padding='same')(pool1)
batch2 = BatchNormalization()(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2), padding='same')(batch2)

flatten = Flatten()(pool2)

# fc1 = Dense(128, activation='relu')(flatten)
# fc1 = Dropout(0.25)(fc1)
# fc2 = Dense(64, activation='relu')(flatten)
# fc2 = Dropout(0.4)(fc2)
fc1 = Dense(32, activation='relu', kernel_initializer='glorot_normal')(flatten)
fc1 = Dropout(0.25)(fc1)

output = Dense(num_classes, activation='softmax')(fc1)

model = Model([inputX, inputY, inputZ], output)

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(lr=learning_rate), metrics=['accuracy']) #beta_1=0.9, beta_2=0.999))

In [ ]:
filepath = "bestWeightsHeterogeneityMixed1D.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', save_best_only=True, mode='max')
callback = [checkpoint]

model.fit([X_train0, X_train1, X_train2], ytrain, epochs=35, batch_size=128, \
          validation_data=([X_test0, X_test1, X_test2], ytest), callbacks=callback)

In [ ]:
model.fit([X_train0, X_train1, X_train2], ytrain, epochs=30, batch_size=128, \
          validation_data=([X_test0, X_test1, X_test2], ytest))#, callbacks=callback)

# LSTM -> Conv1D (Subnet) -> Conv2D

In [ ]:
X_train0 = X_train[:, :, 0].reshape((-1, w_min, 1))
X_train1 = X_train[:, :, 1].reshape((-1, w_min, 1))
X_train2 = X_train[:, :, 2].reshape((-1, w_min, 1))

X_test0 = X_test[:, :, 0].reshape((-1, w_min, 1))
X_test1 = X_test[:, :, 1].reshape((-1, w_min, 1))
X_test2 = X_test[:, :, 2].reshape((-1, w_min, 1))

print (X_train1.shape, X_test1.shape)

In [ ]:
# Subnet X

inputX = Input(shape=(53, 1))
#inputX = Input(shape=(X_train0.shape[1], X_train0.shape[2]))

lstmX = LSTM(32, return_sequences=True)(inputX)

convX1 = Conv1D(filters=8, kernel_size=5, padding='causal', activation='relu')(lstmX)
batchX1 = BatchNormalization()(convX1)
poolX1 = MaxPooling1D(pool_size=2, padding='same')(batchX1)

modelX = poolX1

# Subnet Y

inputY = Input(shape=(53, 1))
#inputY = Input(shape=(X_train1.shape[1], X_train1.shape[2]))

lstmY = LSTM(32, return_sequences=True)(inputY)

convY1 = Conv1D(filters=8, kernel_size=5, padding='causal', activation='relu')(lstmY)
batchY1 = BatchNormalization()(convY1)
poolY1 = MaxPooling1D(pool_size=2, padding='same')(batchY1)

modelY = poolY1

# Subnet Z

inputZ = Input(shape=(53, 1))
#inputZ = Input(shape=(X_train2.shape[1], X_train2.shape[2]))

lstmZ = LSTM(32, return_sequences=True)(inputZ)

convZ1 = Conv1D(filters=8, kernel_size=5, padding='causal', activation='relu')(lstmZ)
batchZ1 = BatchNormalization()(convZ1)
poolZ1 = MaxPooling1D(pool_size=2, padding='same')(batchZ1)

modelZ = poolZ1

In [ ]:
merged_model = merge([modelX, modelY, modelZ], mode='concat')
print (K.int_shape(merged_model))

# final_merge = Reshape((K.int_shape(merged_model)[1]//3, 3, 1))(merged_model)
final_merge = Reshape((K.int_shape(merged_model)[1], K.int_shape(merged_model)[2], 1))(merged_model)
print (K.int_shape(final_merge))

conv1 = Conv2D(filters=8, kernel_size=(3, 3), padding='same')(final_merge)
batch1 = BatchNormalization()(conv1)
pool1 = MaxPooling2D(pool_size=(3, 2), padding='same')(batch1)

conv2 = Conv2D(filters=16, kernel_size=(3, 3), padding='same')(pool1)
batch2 = BatchNormalization()(conv2)
pool2 = MaxPooling2D(pool_size=(3, 2), padding='same')(batch2)

flatten = Flatten()(pool2)

fc1 = Dense(16, activation='relu')(flatten)
fc1 = Dropout(0.25)(fc1)
fc2 = Dense(8, activation='relu')(fc1)
# fc2 = Dropout(0.25)(fc2)
# fc3 = Dense(32, activation='relu')(fc2)
# fc3 = Dropout(0.4)(fc3)

output = Dense(6, activation='softmax')(fc2)

model = Model([inputX, inputY, inputZ], output)

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate), metrics=['accuracy']) #beta_1=0.9, beta_2=0.999))

In [ ]:
filepath = "bestWeightsHeterogeneityMixedLSTM1D.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', save_best_only=True, mode='max')
callback = [checkpoint]

model.fit([X_train0, X_train1, X_train2], ytrain, epochs=15, batch_size=128, \
          validation_data=([X_test0, X_test1, X_test2], ytest), callbacks=callback)

In [ ]:
model.fit([X_train0, X_train1, X_train2], ytrain, epochs=15, batch_size=128, \
          validation_data=([X_test0, X_test1, X_test2], ytest))#, callbacks=callback)

# Conv1D -> LSTM (Subnet) -> Conv2D

In [ ]:
X_train0 = X_train[:, :, 0].reshape((-1, w_min, 1))
X_train1 = X_train[:, :, 1].reshape((-1, w_min, 1))
X_train2 = X_train[:, :, 2].reshape((-1, w_min, 1))

X_test0 = X_test[:, :, 0].reshape((-1, w_min, 1))
X_test1 = X_test[:, :, 1].reshape((-1, w_min, 1))
X_test2 = X_test[:, :, 2].reshape((-1, w_min, 1))

print (X_train1.shape, X_test1.shape)

In [ ]:
# Subnet X

inputX = Input(shape=(X_train0.shape[1], X_train0.shape[2]))

convX1 = Conv1D(filters=8, kernel_size=3, padding='causal', activation='relu')(inputX)
batchX1 = BatchNormalization()(convX1)
poolX1 = MaxPooling1D(pool_size=2, padding='same')(batchX1)

lstmX = LSTM(32, return_sequences=True)(poolX1)


modelX = lstmX

# Subnet Y

inputY = Input(shape=(X_train1.shape[1], X_train1.shape[2]))

convY1 = Conv1D(filters=8, kernel_size=3, padding='causal', activation='relu')(inputY)
batchY1 = BatchNormalization()(convY1)
poolY1 = MaxPooling1D(pool_size=2, padding='same')(batchY1)

lstmY = LSTM(32, return_sequences=True)(poolY1)

modelY = lstmX

# Subnet Z

inputZ = Input(shape=(X_train2.shape[1], X_train2.shape[2]))

convZ1 = Conv1D(filters=8, kernel_size=3, padding='causal', activation='relu')(inputZ)
batchZ1 = BatchNormalization()(convZ1)
poolZ1 = MaxPooling1D(pool_size=2, padding='same')(batchZ1)

lstmZ = LSTM(32, return_sequences=True)(poolZ1)

modelZ = poolZ1

In [ ]:
merged_model = merge([modelX, modelY, modelZ], mode='concat')
print (K.int_shape(merged_model))

# final_merge = Reshape((K.int_shape(merged_model)[1]//3, 3, 1))(merged_model)
final_merge = Reshape((K.int_shape(merged_model)[1], K.int_shape(merged_model)[2], 1))(merged_model)
print (K.int_shape(final_merge))

conv1 = Conv2D(filters=8, kernel_size=(2, 2), padding='same')(final_merge)
batch1 = BatchNormalization()(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2), padding='same')(batch1)

conv2 = Conv2D(filters=16, kernel_size=(2, 2), padding='same')(pool1)
batch2 = BatchNormalization()(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2), padding='same')(batch2)

conv3 = Conv2D(filters=16, kernel_size=(2, 2), padding='same')(pool2)
batch3 = BatchNormalization()(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2), padding='same')(batch3)

flatten = Flatten()(pool3)

fc1 = Dense(16, activation='relu')(flatten)
fc1 = Dropout(0.25)(fc1)
fc2 = Dense(8, activation='relu')(fc1)
# fc2 = Dropout(0.25)(fc2)
# fc3 = Dense(32, activation='relu')(fc2)
# fc3 = Dropout(0.4)(fc3)

output = Dense(num_classes, activation='softmax')(fc2)

model = Model([inputX, inputY, inputZ], output)

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate), metrics=['accuracy']) #beta_1=0.9, beta_2=0.999))

In [ ]:
filepath = "bestWeightsHeterogeneityMixed1DLSTM.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', save_best_only=True, mode='max')
callback = [checkpoint]

model.fit([X_train0, X_train1, X_train2], ytrain, epochs=15, batch_size=128, \
          validation_data=([X_test0, X_test1, X_test2], ytest), callbacks=callback)

In [ ]:
model.fit([X_train0, X_train1, X_train2], ytrain, epochs=15, batch_size=128, \
          validation_data=([X_test0, X_test1, X_test2], ytest))#, callbacks=callback)

In [ ]:
model.fit([X_train0, X_train1, X_train2], ytrain, epochs=15, batch_size=128, \
          validation_data=([X_test0, X_test1, X_test2], ytest))#, callbacks=callback)

# Predictions and Confusion Matrix

In [ ]:
# yPred = model.predict_classes(X_test)
y_prob = model.predict([X_test0, X_test1, X_test2]) 
yPred = y_prob.argmax(axis=-1)

In [ ]:
yTrue = [np.argmax(y) for y in ytest]

In [ ]:
print (yPred[:20])
print (yTrue[:20])

In [ ]:
f1 = f1_score(yTrue, yPred, average = 'weighted')

In [ ]:
print (f1)

In [ ]:
print (classification_report(yTrue, yPred, digits=4))

In [ ]:
conf_matrix = confusion_matrix(yTrue, yPred)#, labels=[0,1,2,3,4,5])
print (conf_matrix)

In [ ]:
df_conf_matrix = pd.DataFrame(conf_matrix, index=list(acts), columns=list(acts))
plt.figure(figsize = (8, 8))
conf_heatmap = sn.heatmap(conf_matrix, annot=True, fmt='g', xticklabels=list(acts), yticklabels=list(acts))
fig = conf_heatmap.get_figure()